In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv



## Importing the required libraries


In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

## Reading the datasets

The datasets has over 5 million rows, working with all the rows at once, might becomne a bit problematic and slow down our device, for this initial prediction, i will be working with 1% of the total training data which should be 500,000 plus rows

In [4]:
#Loading the train_df
#The training data path
train_path = '/kaggle/input/new-york-city-taxi-fare-prediction/train.csv'

#Defining constants
sample_fraction = 0.01
random.seed(92)

#Defining the function to randomly sample 1 percent of the data
def skip_row(idx):
    if idx == 0:
        return False
    return random.random() > sample_fraction

#Loading the df
df = pd.read_csv(train_path, parse_dates = ['pickup_datetime'], skiprows = skip_row)

In [5]:
#Viewing the first 5 rows of the data frame
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3


In [6]:
#Loading the test_df
#The test data path
test_path = '/kaggle/input/new-york-city-taxi-fare-prediction/test.csv'

#Loading the df
test_df = pd.read_csv(test_path, parse_dates = ['pickup_datetime'],)

## Exploring the datasets

In [7]:
#Viewing info about the training data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553345 entries, 0 to 553344
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   key                553345 non-null  object             
 1   fare_amount        553345 non-null  float64            
 2   pickup_datetime    553345 non-null  datetime64[ns, UTC]
 3   pickup_longitude   553345 non-null  float64            
 4   pickup_latitude    553345 non-null  float64            
 5   dropoff_longitude  553344 non-null  float64            
 6   dropoff_latitude   553344 non-null  float64            
 7   passenger_count    553345 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 33.8+ MB


In [8]:
#Statististical summary of the training datset
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,553345.000000,553345.000000,553345.000000,553344.000000,553344.000000,553345.000000
mean,11.358222,-72.509886,39.906591,-72.472268,39.913447,1.684567
std,9.833439,13.800688,10.425359,13.500015,11.529377,1.308479
min,-52.000000,-3313.387292,-3457.346617,-928.225348,-2909.234763,0.000000
25%,6.000000,-73.992037,40.734968,-73.991401,40.734028,1.000000
50%,8.500000,-73.981755,40.752672,-73.980145,40.753132,1.000000
75%,12.500000,-73.966973,40.767177,-73.963604,40.768109,2.000000
max,411.110000,2551.566137,2973.980450,3211.579750,3333.304575,9.000000


In [9]:
#Checking the min and highest date in the training df
df['pickup_datetime'].min(), df['pickup_datetime'].max()

(Timestamp('2009-01-01 00:15:42+0000', tz='UTC'),
 Timestamp('2015-06-30 23:56:38+0000', tz='UTC'))

In [10]:
#info on the test set
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                9914 non-null   object             
 1   pickup_datetime    9914 non-null   datetime64[ns, UTC]
 2   pickup_longitude   9914 non-null   float64            
 3   pickup_latitude    9914 non-null   float64            
 4   dropoff_longitude  9914 non-null   float64            
 5   dropoff_latitude   9914 non-null   float64            
 6   passenger_count    9914 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 542.3+ KB


In [11]:
#Statistical summary of the test data
test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [12]:
#Checking the min and highest date in the test df
test_df['pickup_datetime'].min(), test_df['pickup_datetime'].max()

(Timestamp('2009-01-01 11:04:24+0000', tz='UTC'),
 Timestamp('2015-06-30 20:03:50+0000', tz='UTC'))

#### Observations
* 550k plus rows
* No missing data in the df
* Key column is not neeeded
* There are negative values in the longitude and latitude columns which is unsual
* The passener count column has some unusally large numbers
* Data takes 30+ mb in storage
* Data from both the train and test starts from 2009 and ends in 2015
* The Data contains outliers


## Preparing the data for Training


In [13]:
#Viewing the colums in the df
df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [14]:
#Dropping missng Values
df = df.dropna()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 553344 entries, 0 to 553344
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   key                553344 non-null  object             
 1   fare_amount        553344 non-null  float64            
 2   pickup_datetime    553344 non-null  datetime64[ns, UTC]
 3   pickup_longitude   553344 non-null  float64            
 4   pickup_latitude    553344 non-null  float64            
 5   dropoff_longitude  553344 non-null  float64            
 6   dropoff_latitude   553344 non-null  float64            
 7   passenger_count    553344 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 38.0+ MB


In [16]:
#Splitting the training data into features and labels
X = df.drop(['key', 'fare_amount','pickup_datetime'], axis = 1)
y = df['fare_amount']

In [17]:
#Subsetting the test data
test = test_df.drop(['key','pickup_datetime'], axis = 1)

In [18]:
#Splitting the data into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state = 92)

In [19]:
#Validating the split
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(442675, 5) (110669, 5)
(442675,) (110669,)


## Training a Baseline Model

In [20]:
#Mean Regressor 
class MeanRegressor:
    def fit(self, inputs, targets):
        self.mean = targets.mean()
        
    def predict(self, inputs):
        return np.full(inputs.shape[0], self.mean)

In [21]:
#Instantiating the model
mean_model = MeanRegressor()

#Fitting the model to the input data
mean_model.fit(X_train, y_train)

#Predicting on the train data
train_pred = mean_model.predict(X_train)

#Predicting on the val data
val_pred = mean_model.predict(X_val)


## Evaluating the Baseline Model

In [22]:
#Defining the evaluation function

def rmse(targets, pred):
    return mean_squared_error(targets, pred, squared = False)


In [23]:
#Train rmse
train_rmse = rmse(y_train, train_pred)


#Validation rmse
val_rmse = rmse(y_val, val_pred)

print(f"The Root mean squared error of the baseline model on the train data is: {train_rmse}")
print(f"The Root mean squared erorr of the baseline model on the val data is: {val_rmse}")

The Root mean squared error of the baseline model on the train data is: 9.826516523872696
The Root mean squared erorr of the baseline model on the val data is: 9.861092257690506


### The Hard coded model is off by ~9.8, this is not a good result

## Training a Linear Regression Model

In [24]:
#Instantiating the model
l_reg = LinearRegression()

In [25]:
#Fitting the model to the train data
l_reg.fit(X_train,y_train)

#Making predictions on the train data
train_pred = l_reg.predict(X_train)

#Making predictions on the val data
val_pred = l_reg.predict(X_val)

## Evaluating the Linear Regression Model

In [26]:
#RMSE on the train prediction
train_rmse = rmse(y_train, train_pred)

#RMSE on the val prediction
val_rmse = rmse(y_val, val_pred)

print(f"The RMSE of the model on the train data is: {train_rmse}")
print(f"The RMSE of the model on the val data is: {val_rmse}")

The RMSE of the model on the train data is: 9.824843578269924
The RMSE of the model on the val data is: 9.858982370635724


### The RMSE of the Linear Regression model is not better than what we achieved with our average model

## Predictions for Initial submission to Kaggle

In [27]:
#Making predictions on the test data
test_pred = l_reg.predict(test)

In [28]:
#Preparing the submission file
sub_dir = '/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv'
submission = pd.read_csv(sub_dir)

#Viewing the file
submission

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.35
1,2015-01-27 13:08:24.0000003,11.35
2,2011-10-08 11:53:44.0000002,11.35
3,2012-12-01 21:12:12.0000002,11.35
4,2012-12-01 21:12:12.0000003,11.35
...,...,...
9909,2015-05-10 12:37:51.0000002,11.35
9910,2015-01-12 17:05:51.0000001,11.35
9911,2015-04-19 20:44:15.0000001,11.35
9912,2015-01-31 01:05:19.0000005,11.35


In [29]:
#Subtituting the values
submission['fare_amount'] = test_pred

In [30]:
#Saving the file to csv
submission.to_csv('1st_submission.csv', index = None)

In [31]:
#Defining a function for predicting and saving the submission file
def make_predictions_and_submit(model, test_input, file_name):
    test_pred = model.predict(test_input)
    sub_df = pd.read_csv(sub_dir)
    sub_df['fare_amount'] = test_pred
    sub_df.to_csv(file_name, index = None)
    return sub_df

## Feature Engineering

In [32]:
#Parsing the date column
def parse_date(df, col):
    df[col + '_year'] = df[col].dt.year
    df[col + '_month'] = df[col].dt.month
    df[col + '_day'] = df[col].dt.day
    df[col + '_weekday'] = df[col].dt.weekday
    df[col + '_hour'] = df[col].dt.hour

In [33]:
#Parsing the date column for the training data
parse_date(df, 'pickup_datetime')

In [34]:
#Parsing the date column for the test dat
parse_date(test_df, 'pickup_datetime')

In [35]:
train_df = df.copy()

In [36]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1,2009,3,2,0,20
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6,2015,4,12,6,10
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1,2012,11,6,1,11
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5,2012,10,6,5,15
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3,2015,5,31,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
553340,2010-08-14 21:00:17.0000002,10.9,2010-08-14 21:00:17+00:00,-73.966080,40.805193,-73.954060,40.781668,1,2010,8,14,5,21
553341,2012-04-20 09:31:00.00000039,5.7,2012-04-20 09:31:00+00:00,-73.977957,40.748792,-73.989132,40.742062,1,2012,4,20,4,9
553342,2013-01-21 14:17:17.0000002,7.5,2013-01-21 14:17:17+00:00,-73.999356,40.740943,-73.984007,40.746730,1,2013,1,21,0,14
553343,2013-02-20 08:25:20.0000001,11.5,2013-02-20 08:25:20+00:00,-73.983433,40.738376,-73.975043,40.760101,1,2013,2,20,2,8


In [37]:
#Creating a fucntion for calculating the distance between the pickup and drop off location
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [38]:
#Creating a fucntion for calculating the distance between the pickup and drop off location
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [39]:
train_df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'pickup_datetime_year', 'pickup_datetime_month',
       'pickup_datetime_day', 'pickup_datetime_weekday',
       'pickup_datetime_hour'],
      dtype='object')

In [40]:
#Creating a function to add the distance to the dataframe
def add_distance(df):
    df['trip_distance'] = haversine_np(df['pickup_longitude'],
                                    df['pickup_latitude'],
                                    df['dropoff_longitude'],
                                    df['dropoff_latitude']
                                   )

In [41]:
#Adding trip distance to the train data
add_distance(train_df)
#Adding trip distance to test data
add_distance(test_df)

In [42]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1,2009,3,2,0,20,3.583739
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6,2015,4,12,6,10,3.372594
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1,2012,11,6,1,11,0.564851
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5,2012,10,6,5,15,1.798889
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3,2015,5,31,6,7,6.362372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553340,2010-08-14 21:00:17.0000002,10.9,2010-08-14 21:00:17+00:00,-73.966080,40.805193,-73.954060,40.781668,1,2010,8,14,5,21,2.802987
553341,2012-04-20 09:31:00.00000039,5.7,2012-04-20 09:31:00+00:00,-73.977957,40.748792,-73.989132,40.742062,1,2012,4,20,4,9,1.201860
553342,2013-01-21 14:17:17.0000002,7.5,2013-01-21 14:17:17+00:00,-73.999356,40.740943,-73.984007,40.746730,1,2013,1,21,0,14,1.443436
553343,2013-02-20 08:25:20.0000001,11.5,2013-02-20 08:25:20+00:00,-73.983433,40.738376,-73.975043,40.760101,1,2013,2,20,2,8,2.515395


In [43]:
#Adding distance from popular landmark
#JFK Airport
jfk = -73.7781, 40.6413

#LGA Airport
lga = -73.8740, 40.7769

#EWR Ariport
ewr = -74.175, 40.6895

#MET Museum
met = -74.9632, 40.7794

#World Trade Centre
wtc = -74.0099, 40.7126


In [44]:
#Creating a Function to add the distance from a landmark to the pickup location
def add_landmark_pickup_distance(df, landmark, lonlat):
    lon, lat = lonlat
    df[landmark + 'pickup_distance'] = haversine_np(lon, lat, 
                                                    df['pickup_longitude'], 
                                                    df['pickup_latitude'])

In [45]:
#Creating a Function to add the distance from a landmark to the dropoff location
def add_landmark_dropoff_distance(df, landmark, lonlat):
    lon, lat = lonlat
    df[landmark + 'dropoff_distance'] = haversine_np(lon, lat, 
                                                    df['dropoff_longitude'], 
                                                    df['dropoff_latitude'])

In [46]:
#CReating function to add the distance to the dataframe
def add_landmarks(df):
    landmarks = (('jfk',jfk), ('lga',lga), ('ewr',ewr), ('met',met), ('wtc',wtc))
    for name, lonlat in landmarks:
        add_landmark_pickup_distance(df, name, lonlat)
        add_landmark_dropoff_distance(df, name, lonlat)

In [47]:
#Adding the distance to the train and test dataframe
add_landmarks(train_df)
add_landmarks(test_df)

In [48]:
#Preview of the df
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,...,jfkpickup_distance,jfkdropoff_distance,lgapickup_distance,lgadropoff_distance,ewrpickup_distance,ewrdropoff_distance,metpickup_distance,metdropoff_distance,wtcpickup_distance,wtcdropoff_distance
0,2009-03-02 20:42:49.0000003,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1,2009,3,...,20.983778,23.217742,8.534594,8.227490,18.716496,20.648466,83.392631,83.586331,6.051921,9.378440
1,2015-04-12 10:19:05.0000001,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6,2015,4,...,19.254314,22.406446,10.669770,11.486027,16.913328,15.707058,83.061840,80.762727,2.846452,3.842878
2,2012-11-06 11:33:14.0000004,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1,2012,11,...,21.424882,20.869710,8.354030,8.078787,19.071738,19.206366,83.411924,83.781226,6.718470,6.523576
3,2012-10-06 15:47:00.000000111,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5,2012,10,...,22.704592,23.299184,9.102301,8.497999,18.805216,20.284191,82.558356,83.274480,7.297438,9.091123
4,2015-05-31 07:33:18.0000002,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3,2015,5,...,22.381276,23.168077,10.622476,7.227526,16.686526,22.478479,81.362106,85.011710,4.881942,11.031929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553340,2010-08-14 21:00:17.0000002,10.9,2010-08-14 21:00:17+00:00,-73.966080,40.805193,-73.954060,40.781668,1,2010,8,...,24.131543,21.517253,8.360617,6.757645,21.786297,21.236795,83.937124,84.914825,10.930541,9.000474
553341,2012-04-20 09:31:00.00000039,5.7,2012-04-20 09:31:00+00:00,-73.977957,40.748792,-73.989132,40.742062,1,2012,4,...,20.645286,21.013031,9.290673,10.435612,17.855767,16.709386,82.993887,82.092685,4.838459,3.711827
553342,2013-01-21 14:17:17.0000002,7.5,2013-01-21 14:17:17+00:00,-73.999356,40.740943,-73.984007,40.746730,1,2013,1,...,21.683210,20.934229,11.282836,9.847522,15.860290,17.297871,81.240374,82.496161,3.272393,4.374810
553343,2013-02-20 08:25:20.0000001,11.5,2013-02-20 08:25:20+00:00,-73.983433,40.738376,-73.975043,40.760101,1,2013,2,...,20.389039,21.203192,10.157687,8.706376,17.025295,18.577577,82.595636,83.189992,3.629422,6.039659


In [49]:
#Statistical summary of the train data
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,...,jfkpickup_distance,jfkdropoff_distance,lgapickup_distance,lgadropoff_distance,ewrpickup_distance,ewrdropoff_distance,metpickup_distance,metdropoff_distance,wtcpickup_distance,wtcdropoff_distance
count,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,...,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000,553344.000000
mean,11.358227,-72.509883,39.906589,-72.472268,39.913447,1.684571,2011.743044,6.262087,15.692020,3.039334,...,193.856112,194.035745,183.102192,183.247463,192.100571,192.350423,256.998338,257.206496,179.453261,179.796516
std,9.833447,13.800701,10.425368,13.500015,11.529377,1.308479,1.860347,3.432402,8.683793,1.949327,...,1227.773229,1228.370200,1230.828519,1231.433813,1232.673816,1233.263671,1232.741438,1233.338636,1232.665719,1233.242632
min,-52.000000,-3313.387292,-3457.346617,-928.225348,-2909.234763,0.000000,2009.000000,1.000000,1.000000,0.000000,...,0.303502,0.176746,0.098438,0.061904,0.242005,0.159652,10.487509,13.317465,0.004586,0.004373
25%,6.000000,-73.992037,40.734968,-73.991401,40.734028,1.000000,2010.000000,3.000000,8.000000,1.000000,...,20.570366,20.537289,8.399638,8.340348,16.438220,16.545957,81.854775,81.905525,3.403551,3.645400
50%,8.500000,-73.981755,40.752672,-73.980145,40.753132,1.000000,2012.000000,6.000000,16.000000,3.000000,...,21.213253,21.203691,9.660313,9.575852,17.920710,18.057911,82.667279,82.819691,5.405387,5.565140
75%,12.500000,-73.966973,40.767177,-73.963604,40.768109,2.000000,2013.000000,9.000000,23.000000,5.000000,...,21.913687,21.947435,11.133153,11.120962,19.694564,20.009151,83.872619,84.187480,7.542645,7.826026
max,411.110000,2551.566137,2973.980450,3211.579750,3333.304575,9.000000,2015.000000,12.000000,31.000000,6.000000,...,17105.994493,16422.851685,17105.050438,16429.238357,17131.719095,16405.518246,17182.320582,16370.418478,17118.539583,16416.414769


In [50]:
#Statistical summary of the test data
test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,...,jfkpickup_distance,jfkdropoff_distance,lgapickup_distance,lgadropoff_distance,ewrpickup_distance,ewrdropoff_distance,metpickup_distance,metdropoff_distance,wtcpickup_distance,wtcdropoff_distance
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,...,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273,2011.815816,6.857979,16.194170,2.852834,13.467420,...,20.865709,20.916634,9.739376,9.675325,18.420223,18.584963,83.323284,83.424486,5.763874,6.037548
std,0.042774,0.033541,0.039072,0.035435,1.278747,1.803347,3.353272,8.838482,1.994451,6.868584,...,3.824784,3.303941,3.477757,3.295655,4.336236,4.035394,3.911436,3.501503,4.541681,4.252530
min,-74.252193,40.573143,-74.263242,40.568973,1.000000,2009.000000,1.000000,1.000000,0.000000,0.000000,...,0.417443,0.402060,0.306573,0.285418,11.554724,0.244011,63.764787,62.696395,0.031263,0.040521
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000,2010.000000,4.000000,9.000000,1.000000,8.000000,...,20.559367,20.513155,8.390604,8.311759,16.362588,16.559590,81.812589,81.911529,3.310640,3.669983
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000,2012.000000,7.000000,16.000000,3.000000,15.000000,...,21.187428,21.181275,9.662685,9.478015,17.816546,18.062132,82.603926,82.826881,5.305240,5.541355
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000,2014.000000,10.000000,25.000000,5.000000,19.000000,...,21.873994,21.909607,11.072641,10.965898,19.517192,19.917772,83.788041,84.173738,7.321803,7.757316
max,-72.986532,41.709555,-72.990963,41.696683,6.000000,2015.000000,12.000000,31.000000,6.000000,23.000000,...,135.923926,134.497883,127.435951,126.062625,150.740424,149.428585,194.830920,193.774854,139.971698,138.619586


##### I will be using the range in the test data frame to eliminate outliers from the train_df
* Fare amount: 1 - 500 (This is to remove the negative fair values)
* Passenger count: 1 - 6
* Longitude: -71 - -75
* Latitude: 39 - 42

In [51]:
train_df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'pickup_datetime_year', 'pickup_datetime_month',
       'pickup_datetime_day', 'pickup_datetime_weekday',
       'pickup_datetime_hour', 'trip_distance', 'jfkpickup_distance',
       'jfkdropoff_distance', 'lgapickup_distance', 'lgadropoff_distance',
       'ewrpickup_distance', 'ewrdropoff_distance', 'metpickup_distance',
       'metdropoff_distance', 'wtcpickup_distance', 'wtcdropoff_distance'],
      dtype='object')

In [52]:
#Function for removing outliers
def remove_outliers(df):
    return df[(df['fare_amount'] >= 1)&
              (df['fare_amount'] <= 500)&
              (df['pickup_longitude'] >= -75)&
              (df['pickup_longitude'] <= -71)&
              (df['dropoff_longitude'] >= -75)&
              (df['dropoff_longitude'] <= -71)&
              (df['pickup_latitude'] >= 39)&
              (df['pickup_latitude'] <= 42)&
              (df['dropoff_latitude'] >= 39)&
              (df['dropoff_latitude'] <= 42)&
              (df['passenger_count'] >= 1)&
              (df['passenger_count'] <= 6)]

In [53]:
#Removing the outliers in the train data
train_df = remove_outliers(train_df)

In [54]:
#Saving the processed dataframe
train_df.to_parquet('train.parquet')

## Training and Evaluating Models

In [56]:
#DF Columns
train_df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'pickup_datetime_year', 'pickup_datetime_month',
       'pickup_datetime_day', 'pickup_datetime_weekday',
       'pickup_datetime_hour', 'trip_distance', 'jfkpickup_distance',
       'jfkdropoff_distance', 'lgapickup_distance', 'lgadropoff_distance',
       'ewrpickup_distance', 'ewrdropoff_distance', 'metpickup_distance',
       'metdropoff_distance', 'wtcpickup_distance', 'wtcdropoff_distance'],
      dtype='object')

In [57]:
# Diving the data into features and labels
Xfe = train_df.drop(['key','pickup_datetime','fare_amount'], axis=1)
test_fe = test_df.drop(['key', 'pickup_datetime'], axis=1)
yfe = train_df['fare_amount']

In [58]:
#Saving the porcessed test dataframe
test_fe.to_parquet('test.parquet')

In [59]:
#Subsetting the data into training and val data
X_train, X_val, y_train, y_val = train_test_split(Xfe,yfe, test_size = 0.2, random_state = 92)

In [60]:
#Defining a helper function to help with evaluating the models
def evaluate(model, train_inputs, train_label, val_inputs, val_label):
    train_pred = model.predict(train_inputs)
    train_rmse = rmse(train_label, train_pred)
    val_pred = model.predict(val_inputs)
    val_rmse = rmse(val_label, val_pred)
    return train_pred, train_rmse, val_pred, val_rmse    

### Ridge Regression

In [61]:
#importing the depencies
from sklearn.linear_model import Ridge

In [62]:
#Instanciating the model
model1 = Ridge(random_state=92, alpha = 0.9)

In [63]:
#Fitting the model
model1.fit(X_train, y_train)

Ridge(alpha=0.9, random_state=92)

In [64]:
#Evaluating the model
evaluate(model1, X_train, y_train, X_val, y_val)

(array([50.47619046,  8.21835852, 12.46132378, ..., 10.83306339,
        46.83909149,  8.05468747]),
 5.546323247104452,
 array([15.70091203,  8.23758221,  6.53432886, ...,  6.46515978,
         7.32059719, 23.94851432]),
 5.639419563630839)

In [65]:
#Making Predictions on the test data and saving to csv
make_predictions_and_submit(model1, test_fe,'submission2.csv')

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.463791
1,2015-01-27 13:08:24.0000003,10.945029
2,2011-10-08 11:53:44.0000002,5.959321
3,2012-12-01 21:12:12.0000002,9.258234
4,2012-12-01 21:12:12.0000003,14.630740
...,...,...
9909,2015-05-10 12:37:51.0000002,10.566434
9910,2015-01-12 17:05:51.0000001,12.652854
9911,2015-04-19 20:44:15.0000001,48.931722
9912,2015-01-31 01:05:19.0000005,20.648519


### Random Forest Regressor

In [66]:
#Importing the depnecies
from sklearn.ensemble import RandomForestRegressor

In [67]:
#Creating an instnce
model2 = RandomForestRegressor(random_state = 92, n_jobs = -1, max_depth = 10, n_estimators = 110)

In [68]:
#Fitting the model to data
model2.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, n_estimators=110, n_jobs=-1,
                      random_state=92)

In [69]:
#Evaluating the model
evaluate(model2, X_train, y_train, X_val, y_val)

(array([47.63577601,  7.55058324, 13.35713069, ...,  9.33039664,
        55.5643566 ,  6.41348643]),
 3.6743575132651576,
 array([16.97167219,  9.20519083,  5.01729552, ...,  6.13412765,
         6.2154031 , 22.01129195]),
 4.081526481004471)

In [70]:
#Making Predictions on the test data and saving to csv
make_predictions_and_submit(model2, test_fe,'submission4.csv')

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.433504
1,2015-01-27 13:08:24.0000003,10.783377
2,2011-10-08 11:53:44.0000002,4.948841
3,2012-12-01 21:12:12.0000002,8.138273
4,2012-12-01 21:12:12.0000003,14.110756
...,...,...
9909,2015-05-10 12:37:51.0000002,8.603051
9910,2015-01-12 17:05:51.0000001,13.167858
9911,2015-04-19 20:44:15.0000001,55.408440
9912,2015-01-31 01:05:19.0000005,22.221029


### Gradient Boosting Regressor

In [71]:
#Importing the dependencies
from xgboost import XGBRegressor

In [72]:
#Creating a model instance
model3 = XGBRegressor(random_state = 92, 
                      max_depth = 7,
                      objective = 'reg:squarederror',
                      n_estimators = 200,
                      n_jobs = -1,
                     )

In [73]:
#Fitting the model to data
model3.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=200,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=92,
             reg_alpha=0, reg_lambda=1, ...)

In [74]:
#EValuating the model
evaluate(model3, X_train, y_train, X_val, y_val)

(array([47.631104 ,  8.327806 , 11.579706 , ..., 10.7597275, 53.40587  ,
         6.346895 ], dtype=float32),
 2.582790542630108,
 array([21.065268 , 10.461711 ,  5.541039 , ...,  7.3337455,  5.3362617,
        24.937372 ], dtype=float32),
 4.003510645173853)

In [75]:
#Making Predictions on the test data and saving to csv
make_predictions_and_submit(model3, test_fe,'submission5.csv')

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.911067
1,2015-01-27 13:08:24.0000003,11.999136
2,2011-10-08 11:53:44.0000002,4.497746
3,2012-12-01 21:12:12.0000002,9.122684
4,2012-12-01 21:12:12.0000003,17.539885
...,...,...
9909,2015-05-10 12:37:51.0000002,9.089412
9910,2015-01-12 17:05:51.0000001,10.880651
9911,2015-04-19 20:44:15.0000001,54.111053
9912,2015-01-31 01:05:19.0000005,18.692610
